#   

# 作業目標:

    1. 藉由固定的 dataset, 來驗證不同loss function
    2. Dataset 的特性跟我們選用的loss function 對accrancy 的影響
    
    
# 作業重點: 
    請分別選用 "MSE", "binary _crossentropy"
    查看Train/test accurancy and loss rate
    

# 導入必要的函數

In [1]:
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

Using TensorFlow backend.


# 資料準備

In [2]:
#取得Keras Dataset
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()

In [3]:
#確認 CIFAR10 Dataset 資料維度
print("train data:",'images:',x_img_train.shape,
      " labels:",y_label_train.shape) 
print("test  data:",'images:',x_img_test.shape ,
      " labels:",y_label_test.shape) 

train data: images: (50000, 32, 32, 3)  labels: (50000, 1)
test  data: images: (10000, 32, 32, 3)  labels: (10000, 1)


In [4]:
#資料正規化
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0

In [5]:
#針對Label 做 ONE HOT ENCODE
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
y_label_test_OneHot.shape

(10000, 10)

# 建立模型

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [7]:
model = Sequential()

In [8]:
#卷積層1

In [9]:
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 input_shape=(32, 32,3), 
                 activation='relu', 
                 padding='same'))

In [10]:
model.add(Dropout(rate=0.25))

In [11]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [12]:
#卷積層2與池化層2

In [13]:
model.add(Conv2D(filters=64, kernel_size=(3, 3), 
                 activation='relu', padding='same'))

In [14]:
model.add(Dropout(0.25))

In [15]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [16]:
#建立神經網路(平坦層、隱藏層、輸出層)

In [17]:
model.add(Flatten())
model.add(Dropout(rate=0.25))

In [18]:
model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate=0.25))

In [19]:
model.add(Dense(10, activation='softmax'))

In [20]:
#檢查model 的STACK
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)             

# 載入之前訓練的模型

In [21]:
try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")

載入模型失敗!開始訓練一個新模型


# 訓練模型

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

'''
作業:
請分別選用 "MSE", "binary _crossentropy"
查看Train/test accurancy and loss rate
'''

'\n作業:\n請分別選用 "MSE", "binary _crossentropy"\n查看Train/test accurancy and loss rate\n'

In [23]:
#模型訓練, "Train_History" 把訓練過程所得到的數值存起來
train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)         

#[validation_split = 0.2] validation_split：在0和1之間浮動。用作驗證數據的訓練數據的分數。
#該模型將訓練數據的這一部分分開，不會對其進行訓練，並將在每個時期結束時評估該數據的損失和任何模型指標。
#[batch_size]：整數或None。每個梯度更新的樣本數。指定，batch_size為128


Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 71s 2ms/step - loss: 2.1651 - accuracy: 0.2021 - val_loss: 2.0543 - val_accuracy: 0.2966
Epoch 2/12
37500/37500 [==============================] - 83s 2ms/step - loss: 1.9509 - accuracy: 0.2995 - val_loss: 1.9523 - val_accuracy: 0.3373
Epoch 3/12
37500/37500 [==============================] - 71s 2ms/step - loss: 1.8247 - accuracy: 0.3503 - val_loss: 1.8227 - val_accuracy: 0.3966
Epoch 4/12
37500/37500 [==============================] - 72s 2ms/step - loss: 1.7222 - accuracy: 0.3867 - val_loss: 1.7415 - val_accuracy: 0.4196
Epoch 5/12
37500/37500 [==============================] - 84s 2ms/step - loss: 1.6385 - accuracy: 0.4163 - val_loss: 1.6553 - val_accuracy: 0.4536
Epoch 6/12
37500/37500 [==============================] - 109s 3ms/step - loss: 1.5686 - accuracy: 0.4373 - val_loss: 1.6100 - val_accuracy: 0.4686
Epoch 7/12
37500/37500 [==============================] - 145s 4ms

In [24]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

'''
作業:
請分別選用 "MSE", "binary _crossentropy"
查看Train/test accurancy and loss rate
'''

'\n作業:\n請分別選用 "MSE", "binary _crossentropy"\n查看Train/test accurancy and loss rate\n'

In [25]:
#模型訓練, "Train_History" 把訓練過程所得到的數值存起來
train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)         

#[validation_split = 0.2] validation_split：在0和1之間浮動。用作驗證數據的訓練數據的分數。
#該模型將訓練數據的這一部分分開，不會對其進行訓練，並將在每個時期結束時評估該數據的損失和任何模型指標。
#[batch_size]：整數或None。每個梯度更新的樣本數。指定，batch_size為128

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 77s 2ms/step - loss: 0.2043 - accuracy: 0.9194 - val_loss: 0.2144 - val_accuracy: 0.9143
Epoch 2/12
37500/37500 [==============================] - 72s 2ms/step - loss: 0.2036 - accuracy: 0.9199 - val_loss: 0.2133 - val_accuracy: 0.9149
Epoch 3/12
37500/37500 [==============================] - 112s 3ms/step - loss: 0.2032 - accuracy: 0.9203 - val_loss: 0.2124 - val_accuracy: 0.9153
Epoch 4/12
37500/37500 [==============================] - 93s 2ms/step - loss: 0.2030 - accuracy: 0.9199 - val_loss: 0.2121 - val_accuracy: 0.9152
Epoch 5/12
37500/37500 [==============================] - 77s 2ms/step - loss: 0.2019 - accuracy: 0.9204 - val_loss: 0.2113 - val_accuracy: 0.9158
Epoch 6/12
37500/37500 [==============================] - 79s 2ms/step - loss: 0.2015 - accuracy: 0.9207 - val_loss: 0.2

In [26]:
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])

In [27]:
#模型訓練, "Train_History" 把訓練過程所得到的數值存起來
train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)         

#[validation_split = 0.2] validation_split：在0和1之間浮動。用作驗證數據的訓練數據的分數。
#該模型將訓練數據的這一部分分開，不會對其進行訓練，並將在每個時期結束時評估該數據的損失和任何模型指標。
#[batch_size]：整數或None。每個梯度更新的樣本數。指定，batch_size為128

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 80s 2ms/step - loss: 0.0586 - accuracy: 0.5474 - val_loss: 0.0611 - val_accuracy: 0.5621
Epoch 2/12
37500/37500 [==============================] - 86s 2ms/step - loss: 0.0587 - accuracy: 0.5493 - val_loss: 0.0612 - val_accuracy: 0.5602
Epoch 3/12
37500/37500 [==============================] - 90s 2ms/step - loss: 0.0585 - accuracy: 0.5507 - val_loss: 0.0609 - val_accuracy: 0.5606
Epoch 4/12
37500/37500 [==============================] - 88s 2ms/step - loss: 0.0585 - accuracy: 0.5497 - val_loss: 0.0610 - val_accuracy: 0.5602
Epoch 5/12
37500/37500 [==============================] - 87s 2ms/step - loss: 0.0585 - accuracy: 0.5495 - val_loss: 0.0609 - val_accuracy: 0.5618
Epoch 6/12
37500/37500 [==============================] - 85s 2ms/step - loss: 0.0584 - accuracy: 0.5521 - val_loss: 0.0609 - val_accuracy: 0.5614
Epoch 7/12
37500/37500 [==============================] - 88s 2ms/st